In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import os

In [99]:
# Constant paths
DATA_FOLDER = 'data/'
PICKLES_FOLDER = 'pickles/'
WIKILITE_FOLDER = DATA_FOLDER + 'wiki_lite/'
SUBMISSIONS_FOLDER = 'submissions/'

# Create folders if they don't exist
if not os.path.exists(PICKLES_FOLDER):
    os.makedirs(PICKLES_FOLDER)

if not os.path.exists(SUBMISSIONS_FOLDER):
    os.makedirs(SUBMISSIONS_FOLDER)

In [100]:
# Train data loading
try:
    train_df = pd.read_pickle(PICKLES_FOLDER + 'train_df.pkl')
except:
    train_df = pd.read_csv(DATA_FOLDER + 'train.csv')
    train_df.to_pickle(PICKLES_FOLDER + 'train_df.pkl')
  
# Test data loading  
try:
    test_df = pd.read_pickle(PICKLES_FOLDER + 'test_df.pkl')
except:
    test_df = pd.read_csv(DATA_FOLDER + 'test.csv')  
    test_df.to_pickle(PICKLES_FOLDER + 'test_df.pkl')
   
# Redirects loading
try:
    enwinki_redirects = pd.read_pickle(PICKLES_FOLDER + 'enwiki_redirects.pkl')
except:
    enwinki_redirects = pd.read_csv(WIKILITE_FOLDER + 'enwiki_redirects.tsv', names=['en_title', 'en_redirect_title'], sep='\t')
    enwinki_redirects.to_pickle(PICKLES_FOLDER + 'enwiki_redirects.pkl')
  
# Aliases loading  
try:
    item_aliases = pd.read_pickle(PICKLES_FOLDER + 'item_aliases.pkl')
except:
    item_aliases = pd.read_csv(WIKILITE_FOLDER + 'item_aliases.csv')
    item_aliases.to_pickle(PICKLES_FOLDER + 'item_aliases.pkl')

# Properties loading
try:
    properties = pd.read_pickle(PICKLES_FOLDER + 'property.pkl')
except:
    properties = pd.read_csv(WIKILITE_FOLDER + 'property.csv')
    properties.to_pickle(PICKLES_FOLDER + 'property.pkl')

# Statements loading
try:
    statements = pd.read_pickle(PICKLES_FOLDER + 'statements.pkl')
except:
    statements = pd.read_csv(WIKILITE_FOLDER + 'statements.csv')
    statements.to_pickle(PICKLES_FOLDER + 'statements.pkl')
   
# Wiki items loading 
try:
    wiki_items = pd.read_pickle(PICKLES_FOLDER + 'wiki_items.pkl')
except:
    wiki_items = pd.read_csv(WIKILITE_FOLDER + 'wiki_items.csv')
    wiki_items.to_pickle(PICKLES_FOLDER + 'wiki_items.pkl')

## Data exploration

In [101]:
print('Redirects:')
display(enwinki_redirects)

print('Aliases:')
display(item_aliases)

print('Properties:')
display(properties)

print('Statements:')
display(statements)

print('Wiki items:')
display(wiki_items)

Redirects:


,en_title,en_redirect_title
0,!,Exclamation mark
1,! (CONFIG.SYS directive),CONFIG.SYS
2,! (Donnie Vie Album),Donnie Vie
3,! (Donnie Vie album),Donnie Vie
4,! (The Song Formerly Known As),Unit (album)
...,...,...
8558898,󠁽,Tags (Unicode block)
8558899,󠁾,Tags (Unicode block)
8558900,󠁿,Tags (Unicode block)
8558901,􍁷,Private Use Areas


Aliases:


,item_id,en_alias
0,1,Our Universe
1,1,The Universe
2,1,The Cosmos
3,1,cosmos
4,2,Blue Planet
...,...,...
1495310,76858465,Photinus luciferin 4-monooxygenase (adenosine ...
1495311,76858465,firefly luciferase
1495312,76868858,12 Canum Venaticorum
1495313,76868858,Alpha Canum Venaticorum


Properties:


,property_id,en_label,en_description
0,6,head of government,"head of the executive power of this town, city..."
1,10,video,"relevant video. For images, use the property P..."
2,14,traffic sign,"graphic symbol describing the item, used at th..."
3,15,route map,image of route map at Wikimedia Commons
4,16,highway system,system (or specific country specific road type...
...,...,...,...
6980,7663,Scienza a due voci ID,identifier for an Italian female scientist in ...
6981,7665,FMV World ID,identifier of a video game in the FMV World da...
6982,7666,Visuotinė lietuvių enciklopedija ID,identifier for an item in the online edition o...
6983,7667,Hellenic Civil Aviation Authority airport code,NaN


Statements:


,source_item_id,edge_property_id,target_item_id
0,1,398,497745
1,1,398,1133705
2,1,398,1139177
3,1,398,7439451
4,1,398,15241043
...,...,...,...
26903183,77240068,31,16970
26903184,77240068,131,23556
26903185,77242291,17,145
26903186,77242291,131,23311


Wiki items:


,item_id,en_label,en_description,wikipedia_title
0,1,Universe,totality of space and all contents,Universe
1,2,Earth,third planet from the Sun in the Solar System,Earth
2,3,life,matter capable of extracting energy from the e...,Life
3,4,death,permanent cessation of vital functions,Death
4,5,human,"common name of Homo sapiens, unique extant spe...",Human
...,...,...,...,...
5216231,77042017,HR 4523,NaN,HD 102365
5216232,77043280,Charlie Johnston,NaN,Charlie Johnstone
5216233,77231860,Aldo Rossi,musician,Aldo Rossi (musician)
5216234,77240068,Ebenezer Baptist Church,"church in Atlanta, Georgia, USA",Ebenezer Baptist Church


Merging `item_aliases` and `wiki_items` on `item_id` to get the `wikipedia_title` for each `en_alias`:

In [102]:
merged_wiki_items = wiki_items.merge(item_aliases, how='left', on='item_id')
merged_wiki_items.head()

,item_id,en_label,en_description,wikipedia_title,en_alias
0,1,Universe,totality of space and all contents,Universe,Our Universe
1,1,Universe,totality of space and all contents,Universe,The Universe
2,1,Universe,totality of space and all contents,Universe,The Cosmos
3,1,Universe,totality of space and all contents,Universe,cosmos
4,2,Earth,third planet from the Sun in the Solar System,Earth,Blue Planet


## Trying to find links to test tokens in the wiki items and train tokens

In [103]:
test_df_mod = test_df.copy(deep=True)
train_df_mod = train_df.copy(deep=True)

In [104]:
# Beginning of the URL to wikipedia
URL = 'http://en.wikipedia.org/wiki/'

In [105]:
train_df_mod['full_mention_lower'] = train_df_mod['full_mention'].str.lower()

# We only keep the tokens that have a wiki_url
train_df_mod = train_df_mod[train_df_mod['wiki_url'].notnull() & (train_df_mod['wiki_url'] != '--NME--')]
train_df_mod.head()

,id,token,entity_tag,full_mention,wiki_url,full_mention_lower
3,4,German,B,German,http://en.wikipedia.org/wiki/Germany,german
7,8,British,B,British,http://en.wikipedia.org/wiki/United_Kingdom,british
14,15,BRUSSELS,B,BRUSSELS,http://en.wikipedia.org/wiki/Brussels,brussels
18,19,European,B,European Commission,http://en.wikipedia.org/wiki/European_Commission,european commission
26,27,German,B,German,http://en.wikipedia.org/wiki/Germany,german


In [106]:
test_df_mod['full_mention'] = test_df_mod['full_mention'].str.lower()
test_df_mod.head()

,id,token,entity_tag,full_mention,wiki_url
0,0,-DOCSTART- (947testa CRICKET),NaN,NaN,NaN
1,1,CRICKET,NaN,NaN,NaN
2,2,-,NaN,NaN,NaN
3,3,LEICESTERSHIRE,B,leicestershire,?
4,4,TAKE,NaN,NaN,NaN


For each token in `test_df_mod`, we try to find a link in `train_df_mod` and `merged_wiki_items`:

In [107]:
merged_wiki_items['en_alias_lower'] = merged_wiki_items['en_alias'].str.lower()
merged_wiki_items['wikipedia_title_lower'] = merged_wiki_items['wikipedia_title'].str.lower()

wiki_items['wikipedia_title_lower'] = wiki_items['wikipedia_title'].str.lower()

enwinki_redirects['en_title_lower'] = enwinki_redirects['en_title'].str.lower()

### 1. Just pandas

In [108]:
pandas = False

In [109]:
if pandas:
    aliases_matching = 0
    
    for index, row in tqdm(test_df_mod.iterrows(), total=test_df_mod.shape[0]):
        if str(row['wiki_url']) == 'nan' or row['wiki_url'] != '?':
            continue
            
        token = row['full_mention']
        train_url = train_df_mod[train_df_mod['full_mention_lower'] == token]['wiki_url'].values
        
        if len(train_url) > 0:
            test_df_mod.at[index, 'wiki_url'] = train_url[0]
            
        else:
            wiki_title = merged_wiki_items[(merged_wiki_items['wikipedia_title_lower'] == token) | (merged_wiki_items['en_alias_lower'] == token)]['wikipedia_title'].values
            
            if len(wiki_title) > 0:
                aliases_matching += 1
                test_df_mod.at[index, 'wiki_url'] = URL + wiki_title[0].replace(' ', '_')

### 2. Using dictionaries and redirects

In [110]:
DICT_FOLDER = PICKLES_FOLDER + 'dictionaries/'

In [111]:
try:
    aliases_dict = pickle.load(open(DICT_FOLDER + 'aliases_dict.pkl', 'rb'))
except:
    aliases_dict = pd.Series(merged_wiki_items['wikipedia_title'].values, index=merged_wiki_items['en_alias_lower']).to_dict()
    pickle.dump(aliases_dict, open(DICT_FOLDER + 'aliases_dict.pkl', 'wb'))

In [112]:
try:
    titles_dict = pickle.load(open(DICT_FOLDER + 'titles_dict.pkl', 'rb'))
except:  
    titles_dict = pd.Series(wiki_items['wikipedia_title'].values, index=wiki_items['wikipedia_title_lower']).to_dict()
    pickle.dump(titles_dict, open(DICT_FOLDER + 'titles_dict.pkl', 'wb'))
   

In [113]:
try:
    train_dict = pickle.load(open(DICT_FOLDER + 'train_dict.pkl', 'rb'))
except:
    train_dict = pd.Series(train_df_mod['wiki_url'].values, index=train_df_mod['full_mention_lower']).to_dict()
    pickle.dump(train_dict, open(DICT_FOLDER + 'train_dict.pkl', 'wb'))

In [114]:
try:
    redirects_dict = pickle.load(open(DICT_FOLDER + 'redirects_dict.pkl', 'rb'))
except:
    redirects_dict = pd.Series(enwinki_redirects['en_redirect_title'].values, index=enwinki_redirects['en_title_lower']).to_dict()
    pickle.dump(redirects_dict, open(DICT_FOLDER + 'redirects_dict.pkl', 'wb'))

### Saving all the dictionaries as pickles

In [115]:
dictionaries = True

In [116]:
if dictionaries:
    aliases_matching = 0
    redirects_matching = 0
    
    for index, row in tqdm(test_df_mod.iterrows(), total=test_df_mod.shape[0]):
        if str(row['wiki_url']) == 'nan' or row['wiki_url'] != '?':
            continue
            
        token = row['full_mention']
        train_url = train_dict.get(token)
        
        
        if train_url is not None:
            # We found a link in the train data and it is the true identity so we can use it
            test_df_mod.at[index, 'wiki_url'] = train_url
            continue
            
        else:
            wiki_title = titles_dict.get(token)
            
            if wiki_title is not None:
                aliases_matching += 1
                
            else:
                wiki_title = aliases_dict.get(token)
                
                if wiki_title is not None:
                    aliases_matching += 1
                    
        if wiki_title is not None:
            redirect_title = redirects_dict.get(wiki_title.lower())    

            if redirect_title is not None:
                redirects_matching += 1
                test_df_mod.at[index, 'wiki_url'] = URL + redirect_title.replace(' ', '_')

            else:
                test_df_mod.at[index, 'wiki_url'] = URL + wiki_title.replace(' ', '_')
            
            test_df_mod.at[index, 'wiki_url'] = URL + wiki_title.replace(' ', '_')


100%|██████████| 104890/104890 [00:02<00:00, 38224.11it/s]


### Let's look at the results

In [117]:
aliases_matching

2107

In [118]:
redirects_matching

393

Let's see how many tokens we found links for:

In [119]:
print(f"Previously we had {test_df[test_df['wiki_url'] == '?']['wiki_url'].count()} tokens without a link")
print(f"Now we have {test_df_mod[test_df_mod['wiki_url'] == '?']['wiki_url'].count()} tokens without a link")

Previously we had 9166 tokens without a link
Now we have 547 tokens without a link


### Creating submission file for second part

In [120]:
submission = test_df_mod[['id', 'wiki_url']]
submission.loc[:, 'wiki_url'] = submission['wiki_url'].apply(lambda x: 'NOT_FOUND' if not (str(x).startswith('http') or str(x) == '?') else x)
# submission.loc[:, 'wiki_url'] = submission['wiki_url'].apply(lambda x: 'NOT_FOUND' if not (str(x).startswith('http')) else x)

name = 'tr_title_alias_dict_redirects_url'

submission.to_csv(SUBMISSIONS_FOLDER + name + '.csv', index=False)

### Looking at differences in results

In [121]:
name_1 = 'tr_title_alias_pd'
name_2 = name

df1 = pd.read_csv(SUBMISSIONS_FOLDER + name_1 + '.csv')
df2 = pd.read_csv(SUBMISSIONS_FOLDER + name_2 + '.csv')

mask = df1['wiki_url'] != df2['wiki_url']

diff = pd.merge(df1[mask], df2[mask], how='inner', on='id', suffixes=('_pd', '_dict'))
diff

,id,wiki_url_pd,wiki_url_dict
0,69,http://en.wikipedia.org/wiki/Surrey_County_Cri...,http://en.wikipedia.org/wiki/Surrey
1,264,http://en.wikipedia.org/wiki/Surrey_County_Cri...,http://en.wikipedia.org/wiki/Surrey
2,315,http://en.wikipedia.org/wiki/Surrey_County_Cri...,http://en.wikipedia.org/wiki/Surrey
3,519,http://en.wikipedia.org/wiki/Chester-le-Street,http://en.wikipedia.org/wiki/Riverside_Ground
4,578,http://en.wikipedia.org/wiki/Surrey_County_Cri...,http://en.wikipedia.org/wiki/Surrey
...,...,...,...
1018,104572,http://en.wikipedia.org/wiki/Seville,http://en.wikipedia.org/wiki/Sevilla_FC
1019,104590,http://en.wikipedia.org/wiki/Heracles,http://en.wikipedia.org/wiki/Hercules
1020,104617,http://en.wikipedia.org/wiki/The_Irishman_(197...,http://en.wikipedia.org/wiki/The_Irishman
1021,104819,http://en.wikipedia.org/wiki/1998_FIFA_World_Cup,http://en.wikipedia.org/wiki/Cricket_World_Cup
